<a href="https://colab.research.google.com/github/rostro36/Partisan-Responses/blob/master/06_GraphWriter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Partisan-Responses-master')

Install all requirements.

In [ ]:
!pip install -r requirements.txt
!pip install torchtext -U
!pip install spacy==2.1.0
!pip install neuralcoref allennlp hnswlib allennlp-models
!python -m spacy download en

Import everything

In [ ]:
import neuralcoref
import spacy
import re
import pickle
import pandas as pd
import gc
from Answer import Answer
import utils

Setup everything for coref

In [ ]:
nlp = spacy.load('en')
neuralcoref.add_to_pipe(nlp)

identifier='coref'
last_check=0
step_size=1000

def load_pickles(identifier, checkpoint):
  if os.path.exists("df"+identifier+str(checkpoint)+".pickle"):
    df = pd.read_pickle("df"+identifier+str(checkpoint)+".pickle")
  return df

def dump_pickles(identifier, checkpoint, df):
  df.to_pickle("df"+identifier+str(checkpoint)+".pickle")
  gc.collect()
  return 

def make_corefs(content):
    content=str(re.sub("\.(?=\s[a-z0-9]|\sI[\W\s])", ",", content))
    doc=nlp(content)
    return doc._.coref_resolved

df = pd.read_pickle("final_data.pkl")

Do coref

In [ ]:
for i in range(last_check+1,int(df.shape[0]/step_size)):
  print(i)
  for j in range(step_size):
    df.iloc[step_size*(i-1)+j]['answer']=make_corefs(df.iloc[step_size*(i-1)+j]['answer'])
  dump_pickles(identifier,i,df)
print('done with ordered')
for j in range(df.shape[0]-int(df.shape[0]/step_size)*step_size):
  df.iloc[step_size*int(df.shape[0]/step_size)+j]['answer']=make_corefs(df.iloc[step_size*int(df.shape[0]/step_size)+j]['answer'])
df.to_pickle(identifier+".pkl")

Setup for parsing

In [ ]:
identifier=identifier
last_check=1
step_size=100
#df is still from the cell above above

def parse_entry(entry,verb_dict,verb_list):
    result=dict()
    result['question']=' '.join([token.text for token in utils.sp(entry['question'])])
    phrase_corpus, triplet_id, parsed_text,parsed=Answer(entry['answer']).create_training(verb_dict,verb_list)
    result['corpus']=' ; '.join(phrase_corpus)
    result['tags']=' '.join(['<phrase>']*len(phrase_corpus))
    result['triplet_id']=' ; '.join([re.sub('\,','',str(x))[1:-1] for x in triplet_id])
    result['parsed_text']=parsed_text
    result['parsed']=' '.join([str(x) for x in parsed])
    return result

def load_pickles(identifier, checkpoint):
  if os.path.exists("verb_dict"+identifier+str(checkpoint)+".pickle"):
    with open('verb_dict'+identifier+str(checkpoint)+'.pickle', 'rb') as handle:
      verb_dict = pickle.load(handle)
    with open('verb_list'+identifier+str(checkpoint)+'.pickle', 'rb') as handle:
      verb_list = pickle.load(handle)
    with open('result'+identifier+str(checkpoint)+'.pickle', 'rb') as handle:
      result = pickle.load(handle)
  else:
    print('File does not exist yet.')
    verb_dict=dict()
    verb_list=[]
    result=[]
  return verb_dict, verb_list, result

def dump_pickles(identifier, checkpoint, verb_dict, verb_list, result):
  with open("verb_dict"+identifier+str(checkpoint)+".pickle", 'wb') as handle:
      pickle.dump(verb_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
  with open("verb_list"+identifier+str(checkpoint)+".pickle", 'wb') as handle:
      pickle.dump(verb_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
  with open("result"+identifier+str(checkpoint)+".pickle", 'wb') as handle:
      pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)
  return 

verb_dict, verb_list, result=load_pickles(identifier, last_check)

Process data

In [ ]:
for i in range(last_check+1,int(df.shape[0]/step_size)):
  print(i)
  df[step_size*(i-1):step_size*i].apply(lambda x: result.append(parse_entry(x,verb_dict,verb_list)), axis=1)
  dump_pickles(identifier,i,verb_dict,verb_list,result)
print('done with ordered')
df[step_size*i:].apply(lambda x: result.append(parse_entry(x,verb_dict,verb_list)), axis=1)
dump_pickles(identifier,1+df.shape[0]/step_size,verb_dict,verb_list,result)

Post-processing of data

In [ ]:
df=pd.DataFrame(result) 
df=df.sample(frac=1, random_state=36)
df=df.dropna(axis=0, how='any')
df=df[df.iloc[:,4].apply(lambda x: len(x)<1000)]
df=df[df.iloc[:,1].apply(lambda x: len(x)<1000)]

Write to variables

In [ ]:
identifier='../GraphWriter-master/data/preprocessed'
m=df.iloc[:50]
m.to_csv(identifier+'.control.tsv', sep='\t', index=False, header=False)
m=df.iloc[50:1050]
m.to_csv(identifier+'.val.tsv', sep='\t', index=False, header=False)
m=df.iloc[1050:2050]
m.to_csv(identifier+'.test.tsv', sep='\t', index=False, header=False)
m=df.iloc[2050:]
m.to_csv(identifier+'.train.tsv', sep='\t', index=False, header=False)

with open(identifier+'.vocab', 'w') as filehandle:
    filehandle.writelines("%s\n" % verb.upper() for verb in verb_list)

Train with GraphWriter

In [ ]:
!python ../GraphWriter-master/train.py -save ../GraphWriter-master/partisanResponses

Use file with lowest vloss in the folder "partisanResponses" of GraphWriter-master to generate responses on test data.


E.g. in the next cell "9.vloss-3.980539.lr-0.1" was used.

In [ ]:
!python ../GraphWriter-master/generator.py -save=../GraphWriter-master/partisanResponses/9.vloss-3.980539.lr-0.1